In [1]:
"""
Build /mnt/ebs/code/forked-context-aware-decoding/eval/apps_tmp/random_2_-1.jsonl with apps data.
Where the with-context prompt would be containing random training examples
with their ground-truth code.
"""

'\nBuild /mnt/ebs/code/forked-context-aware-decoding/eval/apps_tmp/random_2_-1.jsonl with apps data.\nWhere the with-context prompt would be containing random training examples\nwith their ground-truth code.\n'

In [2]:
import pandas as pd

In [3]:
def load_data(dataset, data_type):
    data_path_1 = f"/mnt/ebs/code/hallucination-mitigation/data/{dataset}/{data_type}_samples_1.jsonl"
    data_path_2 = f"/mnt/ebs/code/hallucination-mitigation/data/{dataset}/{data_type}_samples_2.jsonl"
    df_1 = pd.read_json(data_path_1, lines=True)
    df_2 = pd.read_json(data_path_2, lines=True)
    # combine the two dataframes
    df = pd.concat([df_1, df_2], ignore_index=True)
    return df

In [4]:
train_df = load_data("apps", "train")
test_df = load_data("apps", "test")

In [16]:
num_shots = 2

In [ ]:
def build_dsc_context(problem):
    prompt = "\nQUESTION:\n"
    prompt += problem["question"]
    fn_name = problem["fn_name"]

    if not fn_name:
        call_format = "\nPlease write your code using Standard Input, i.e. input() and print()."
        prompt += call_format
    else:
        call_format = "\Please write your code using Call-Based format."
        prompt += call_format

    system_prompt = "You are an expert code developer with years of experience."
    user_prompt = f'''As an expert code developer with years of experience, please provide the python code based on the question. Ensure the code is enclosed within triple backticks (```) to mark the start and end of the code block.\n{prompt}'''

    prompt = f'''You are an AI programming assistant, utilizing the DeepSeek Coder model, developed by DeepSeek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer.
### Instruction:
{system_prompt}

{user_prompt}
### Response:
'''
    return prompt

In [ ]:
output = []

for i in range(10):
    test_problem = test_df.iloc[i]
    train_problems = train_df.sample(n=num_shots)

    line_with_context = {
        "input_index": test_problem["task_id"],
        "assigned_model": "deepseek-ai/deepseek-coder-6.7b-base",
        "assigned_process": 0,
        "filter_p": 1,
        "context_string": None, # TODO
        "assigned_weight": 2
        # Note: I do not give gold answers here because I do not use it for evaluation.
        # I could also always access it using task_id.
    }

    line_without_context = {
        "input_index": test_problem["task_id"],
        "assigned_model": "deepseek-ai/deepseek-coder-6.7b-base",
        "assigned_process": 1,
        "context_string": build_dsc_context(test_problem),
        "assigned_weight": -1
    }

In [6]:
test_problem = test_df.iloc[0]

In [14]:
prompt = build_dsc_context(test_problem)
print(prompt)

You are an AI programming assistant, utilizing the DeepSeek Coder model, developed by DeepSeek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer.
### Instruction:
You are an expert code developer with years of experience.

As an expert code developer with years of experience, please provide the python code based on the question. Ensure the code is enclosed within triple backticks (```) to mark the start and end of the code block.

QUESTION:
An accordion is a string (yes, in the real world accordions are musical instruments, but let's forget about it for a while) which can be represented as a concatenation of: an opening bracket (ASCII code $091$), a colon (ASCII code $058$), some (possibly zero) vertical line characters (ASCII code $124$), another colon, and a closing bracket (ASCII code $093$). The length of the accordion is the number

In [17]:
train_problems = train_df.sample(n=num_shots)

In [21]:
train_problems.iloc[1]

task_id                                                      4642
fn_name                                          calculate_damage
input_sample                             ['grass', 'fire', 35, 5]
question        It's a Pokemon battle! Your task is to calcula...
solutions       [import math\n\neffectiveness = {\n    "electr...
difficulty                                           introductory
url             https://www.codewars.com/kata/536e9a7973130a06...
Name: 4642, dtype: object